In [6]:
import os

In [7]:
%pwd

'c:\\Users\\DELL\\OneDrive\\Desktop\\Projects\\Projects\\Machine learning\\Red_wine_project\\research'

In [8]:
os.chdir("../")

In [9]:
%pwd

'c:\\Users\\DELL\\OneDrive\\Desktop\\Projects\\Projects\\Machine learning\\Red_wine_project'

In [ ]:
# entity
from dataclasses import dataclass  
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir:Path 
    data_path:Path

In [11]:
#config manager
from Red_Wine_Prediction.constants import * 

from Red_Wine_Prediction.utils.common import read_yaml,create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH ,
        schema_filepath= SCHEMA_FILE_PATH ):
        
            self.config =read_yaml(config_filepath)
            self.params =read_yaml(params_filepath)
            self.schema =read_yaml(schema_filepath)
            
            create_directories([self.config.artifacts_root])
            
            
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config =self.config.data_transformation 
        
        create_directories([config.root_dir])
        
        Data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            
            
        )
        
        return Data_transformation_config
    

In [13]:
import pandas as pd

In [ ]:
# #component 

# import os 
# from Red_Wine_Prediction import logger 
# from sklearn.model_selection import train_test_split 
# import pandas as pd 


# class DataTransformation:
    
#     def __init__(self,config:DataTransformationConfig):
#         self.config = config 
        

        
        
        
        
#     def train_test_splitting(self):
#         data=pd.read_csv(self.config.data_path) 
        
        
        
        
        
#         #Split the data into 75,25
#         train, test =train_test_split(data)
        
#         train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
#         test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index=False)
        
#         logger.info("Splitted data  into training and testing sets")
#         logger.info(train.shape)
#         logger.info(test.shape)
        
#         print(train.shape)
#         print(test.shape)

In [ ]:
# component

import os
import numpy as np
import pandas as pd
import joblib

from Red_Wine_Prediction import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


class DataTransformation:
    
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.scaler = StandardScaler()
        
        
    def train_test_splitting(self):
        # -----------------------------
        # Load data
        # -----------------------------
        data = pd.read_csv(self.config.data_path)
        logger.info(f"Data loaded from {self.config.data_path}")

        # -----------------------------
        # Feature Engineering
        # -----------------------------
        data["log_residual_sugar"] = np.log1p(data["residual sugar"])
        data["log_chlorides"] = np.log1p(data["chlorides"])
        data["log_free_sulfur_dioxide"] = np.log1p(data["free sulfur dioxide"])
        data["log_total_sulfur_dioxide"] = np.log1p(data["total sulfur dioxide"])
        data["log_sulphates"] = np.log1p(data["sulphates"])
        data["alcohol_density_ratio"] = data["alcohol"] / (data["density"] + 1e-6)

        logger.info("Feature engineering completed")

        # -----------------------------
        # Separate features & target
        # -----------------------------
        X = data.drop(columns=["quality"])
        y = data["quality"]

        # -----------------------------
        # Train-Test Split (75-25)
        # -----------------------------
        X_train, X_test, y_train, y_test = train_test_split(
            X,
            y,
            test_size=0.25,
            random_state=42
        )

        # -----------------------------
        # Scaling (NO data leakage)
        # -----------------------------
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

        # Re-attach target
        train = pd.concat([X_train_scaled, y_train.reset_index(drop=True)], axis=1)
        test = pd.concat([X_test_scaled, y_test.reset_index(drop=True)], axis=1)

        # -----------------------------
        # Save artifacts
        # -----------------------------
        os.makedirs(self.config.root_dir, exist_ok=True)

        train_path = os.path.join(self.config.root_dir, "train.csv")
        test_path = os.path.join(self.config.root_dir, "test.csv")
        scaler_path = os.path.join(self.config.root_dir, "scaler.joblib")

        train.to_csv(train_path, index=False)
        test.to_csv(test_path, index=False)
        joblib.dump(self.scaler, scaler_path)

        # -----------------------------
        # Logging
        # -----------------------------
        logger.info("Splitted data into training and testing sets")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")
        logger.info(f"Scaler saved at: {scaler_path}")

        print(train.shape)
        print(test.shape)


In [ ]:
data=pd.read_csv('data\winequality-red.csv')



In [17]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [19]:
import  numpy as np

In [ ]:
data["log_residual_sugar"] = np.log1p(data["residual sugar"])
data["log_chlorides"] = np.log1p(data["chlorides"])
data["log_free_sulfur"] = np.log1p(data["free sulfur dioxide"])
data["log_total_sulfur"] = np.log1p(data["total sulfur dioxide"])
data["log_sulphates"] = np.log1p(data["sulphates"])




In [21]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,log_residual_sugar,log_chlorides,log_free_sulfur,log_total_sulfur,log_sulphates
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023,1.218131,0.083038,2.639013,3.634750,0.501073
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569,0.269969,0.038991,0.623790,0.682575,0.093731
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000,0.641854,0.011929,0.693147,1.945910,0.285179
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000,1.064711,0.067659,2.079442,3.135494,0.438255
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000,1.163151,0.076035,2.708050,3.663562,0.482426
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000,1.280934,0.086178,3.091042,4.143135,0.548121
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000,2.803360,0.476855,4.290459,5.669881,1.098612


density(0.001887) already includes alcohol, and this ratio separates alcohol’s real impact on quality.

In [22]:
data["alcohol_density_ratio"] = data["alcohol"] / data["density"]

In [16]:
try:
    config =ConfigurationManager()
    data_transformation_config= config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
    
except Exception as e:
    raise e

[2026-01-03 20:00:52,391: INFO: common: YAML file loaded successfully: config\config.yaml]
[2026-01-03 20:00:52,394: INFO: common: YAML file loaded successfully: params.yaml]
[2026-01-03 20:00:52,397: INFO: common: YAML file loaded successfully: schema.yaml]
[2026-01-03 20:00:52,399: INFO: common: Directory created at: artifacts]
[2026-01-03 20:00:52,400: INFO: common: Directory created at: artifacts/data_transformation]
[2026-01-03 20:00:52,422: INFO: 3010335155: Splitted data  into training and testing sets]
[2026-01-03 20:00:52,423: INFO: 3010335155: (1199, 12)]
[2026-01-03 20:00:52,424: INFO: 3010335155: (400, 12)]
(1199, 12)
(400, 12)
